# Lab 12-3 many to many fixed
### hihello training 
* many to many
* fixed input sequence length

In [1]:
import tensorflow as tf
from pprint import pprint
tf.set_random_seed(777)

### Prepairing dataset
hih -> ihe, ihe -> hel, hel -> ell, ell -> llo

In [2]:
strings = 'hihello'
char_set = sorted(list(set('hihello' + 'hello')))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

In [3]:
data = []
for i in range(len(strings) -2):
    data.append(strings[i:i+3])

x_strings = data[0:4]
y_strings = data[1:5]

print(x_strings, y_strings)

['hih', 'ihe', 'hel', 'ell'] ['ihe', 'hel', 'ell', 'llo']


### Preprocessing dataset

In [4]:
x_data = list(map(lambda strings : [char2idx.get(char) for char in strings], x_strings))
y_data = list(map(lambda strings : [char2idx.get(char) for char in strings], x_strings))

# data 형태확인
print(x_data)
print(y_data)

[[1, 2, 1], [2, 1, 0], [1, 0, 3], [0, 3, 3]]
[[1, 2, 1], [2, 1, 0], [1, 0, 3], [0, 3, 3]]


### Creating graph

In [5]:
# input
data = tf.data.Dataset.from_tensor_slices((x_data, y_data))
data = data.shuffle(buffer_size = 4)
data = data.batch(batch_size = 2)
iterator = data.make_initializable_iterator()
x_mb, y_mb = iterator.get_next()

In [6]:
# hyper-parameters for lstm (many to many), one-hot encoding
n_of_classes = len(idx2char)
hidden_size = 10

one_hot_encoding = tf.eye(num_rows = n_of_classes, dtype = tf.float32)
one_hot_encoding = tf.get_variable(name = 'one_hot_encoding',
                                   initializer = one_hot_encoding, trainable = False)

x_mb_batch = tf.nn.embedding_lookup(params = one_hot_encoding, ids = x_mb)
lstm_cell = tf.nn.rnn_cell.LSTMCell(num_units = hidden_size, dtype = tf.float32)
score_cell = tf.contrib.rnn.OutputProjectionWrapper(cell = lstm_cell, output_size = n_of_classes)
outputs, _ = tf.nn.dynamic_rnn(cell = score_cell, inputs = x_mb_batch, dtype = tf.float32)

In [7]:
weights = tf.ones_like(tensor = x_mb, dtype = tf.float32)
s2s_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = y_mb, weights = weights)
prediction = tf.argmax(input = outputs, axis = -1)

In [8]:
# training
lr = .1
opt = tf.train.AdamOptimizer(learning_rate = lr)
training_op = opt.minimize(loss = s2s_loss)

### Training

In [9]:
sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config = sess_config)
sess.run(tf.global_variables_initializer())

In [10]:
epochs = 10
tr_loss_hist = []

for epoch in range(epochs):
    
    sess.run(iterator.initializer)
    avg_tr_loss = 0
    step = 0
    
    try:
        while True:
            _, loss = sess.run([training_op, s2s_loss])
            avg_tr_loss += loss
            step += 1
    except:
        avg_tr_loss /= step
        tr_loss_hist.append(avg_tr_loss)
        
    print('epoch : {:3}, loss : {:.3f}'.format(epoch + 1, avg_tr_loss))

epoch :   1, loss : 1.581
epoch :   2, loss : 1.221
epoch :   3, loss : 0.820
epoch :   4, loss : 0.449
epoch :   5, loss : 0.208
epoch :   6, loss : 0.086
epoch :   7, loss : 0.033
epoch :   8, loss : 0.012
epoch :   9, loss : 0.005
epoch :  10, loss : 0.003


### Accuracy

In [11]:
yhat = sess.run(prediction, feed_dict = {x_mb : x_data})

In [12]:
print('true : {}, \n prediction : {}'.format(y_data, yhat))

true : [[1, 2, 1], [2, 1, 0], [1, 0, 3], [0, 3, 3]], 
 prediction : [[1 2 1]
 [2 1 0]
 [1 0 3]
 [0 3 3]]
